In [1]:
import json
import random
import csv
import torch
import numpy as np
from numpy import mean
from tqdm.notebook import tqdm

In [47]:
def read_answer(path):
    with open(path, 'rb') as f:
        answer = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in tqdm(answer['data']):
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                temp_answer = []
                for answer in qa['answers']:
                    temp_answer.append(answer['text'])
                if len(temp_answer) != 0: # answers의 길이가 0 == 답변할 수 없는 질문
                    contexts.append(context)
                    questions.append(question)
                    answers.append(temp_answer)

    return contexts, questions, answers

In [51]:
test_contexts, test_questions, test_answers = read_answer("./datasets2/klue-mrc-v1.1_dev.json")
print(len(test_answers))


4008


In [52]:
name = 'kobigbird_v2_ep2_max1024_lr6e-05_206_00'
with open(f'./submissions/{name}.csv','r',encoding='utf-8') as f:
    rdr = csv.reader(f)
    prediction = []
    for line in rdr:
        prediction.append(line[1])

del prediction[0]
prediction[0]

'740Li 25주년 에디션'

In [53]:
def calc_distance(a, b):
    ''' 레벤슈타인 거리 계산하기 '''
    if a == b:
        return 0 # 같으면 0을 반환
    
    a_len = len(a) # a 길이
    b_len = len(b) # b 길이
    if a == "":
        return b_len
    if b == "":
        return a_len
    
    matrix = [[] for i in range(a_len+1)]
    
    for i in range(a_len+1): # 0으로 초기화
        matrix[i] = [0 for j in range(b_len+1)]
        
    # 0일 때 초깃값을 설정
    for i in range(a_len+1):
        matrix[i][0] = i
        
    for j in range(b_len+1):
        matrix[0][j] = j
        
    # 표 채우기 --- (※2)
    for i in range(1, a_len+1):
        ac = a[i-1]
        for j in range(1, b_len+1):
            bc = b[j-1]
            cost = 0 if (ac == bc) else 1
            matrix[i][j] = min([
                matrix[i-1][j] + 1,     # 문자 삽입
                matrix[i][j-1] + 1,     # 문자 제거
                matrix[i-1][j-1] + cost # 문자 변경
            ])
    return matrix[a_len][b_len]

In [54]:
print(calc_distance(prediction[0], test_answers[0][0]))
print(prediction[0], test_answers[0][0], sep='\n')

2
740Li 25주년 에디션
뉴 740Li 25주년 에디션


In [55]:
score = []
for i in range(int(len(test_answers))):
    s = calc_distance(prediction[i], test_answers[i][0])
    score.append(s)
mean(score)

2.1037924151696608